# File Setup

In [9]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import netCDF4
# import cartopy
import math
import sys
import os
import itertools
import scipy.stats
import json

ModuleNotFoundError: No module named 'legate'

In [7]:

!$sys.executable -m pip install legate


# Load Pacific SLR Data

In [3]:
# Site ID 1 (index 0) is Tarawa

## Time series, full distributions

In [4]:
# List all AISs used
AISs = [
    "k14",
    "dp16",
    "dp21",
    "k14v",
    "dp16v",
    "dp21v"    
]

# Create a dictionary to store the dictionaries of probabilities in
probs_dict = {}

# Loop over each AIS type
for AIS in AISs:
    # Create list of rcp scenarios to loop over
    rcps = [
        "26",
        "45",
        "85"
    ]

    # loop over each rcp
    for rcp in rcps:

        # get file name 
        file = "pac_islands/workflows/wf_{}/rcp{}/total-workflow_localsl.nc".format(AIS,rcp,rcp)

        # Load the data
        ds = netCDF4.Dataset(file)

        # Extract the variables
        time = np.array(ds.variables['years'])
        lat = np.array(ds.variables['lat'])
        long = np.array(ds.variables['lon'])
        localSL_quantiles = np.array(ds.variables['localSL_quantiles'])
        quantiles = np.array(ds.variables['quantiles'])
        site_id = np.array(ds.variables['id'])

        for year,year_index in zip(time,np.arange(0,len(time),1)):
            var = localSL_quantiles[:,0,year_index]
            var_mean, var_stdev = np.mean(var),np.std(var)
            zscores = [(x-var_mean)/var_stdev for x in var]
            p_values = [scipy.stats.norm.pdf(abs(x))*2 for x in zscores]
            p_values = p_values/np.sum(p_values) #normalise the pvalues
            SLR = {str(x):str(y) for x,y in zip(p_values,localSL_quantiles[:,0,year_index])}

            probs_dict.update({
                str((AIS,rcp,year)):SLR
            })


In [5]:
with open('distributions_dict.json', 'w') as fp:
    json.dump(probs_dict, fp)

In [6]:
ds

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    description: Total SLR for workflow
    history: Created Tue May  4 15:10:51 2021; Seed Value 1122
    source: FACTS: Post-processed total among available contributors: verticallandmotion-kopp14-verticallandmotion_localsl.nc,icesheets-kopp14SROCC-icesheets-rcp85_GIS_localsl.nc,glaciers-kopp14-glaciers-rcp85_localsl.nc,icesheets-dp20-icesheet-rcp85_AIS_localsl.nc,oceandynamics-kopp14-oceandynamics-rcp85_localsl.nc,landwaterstorage-ssp-landwaterstorage-rcp85_localsl.nc
    dimensions(sizes): nsites(6), years(14), quantiles(108)
    variables(dimensions): float32 lat(nsites), float32 lon(nsites), int32 id(nsites), int32 years(years), float32 quantiles(quantiles), int16 localSL_quantiles(quantiles, nsites, years)
    groups: 